# Basics of mobile robotics [MICRO-452]  
## Project report, EPFL Robotic Master, 12.12.21

<img src="image/logo-epfl.png" alt="Drawing" style="width: 200px;"/>

Author: **Nour Tnani [296442], Xavier Nal [288275], Alicia Mauroux [274618], Antoine Perrin [283652]**

## 0. Introduction
Our project is to find a parking spot to our Thymio and to guide it to this spot using Global Navigation. We imagined it in a whole city like in this figure:
<img src="image/map_multi.jpg" alt="Drawing" style="width: 200px;"/>

But for this project, we focused only on a part of the map like you can see in the figure2. Thymio will evolve on this map where there is two houses and four parking spots. Three of this spots are occupied and only one of them is free. There will be some "dumb" Thymio that will evolve in this map too, our Thymio have to avoid them, using Local Navigation. 

Here is our map:

Figure2: Green = parking spots, Blue = obstacles, Black = "dumb" Thymio's path
<img src="image/map.jpg" alt="Drawing" style="width: 400px;"/>

For this project, we chose to cut the work in 4 parts:
- **Current goal update** [Global Navigation]
- **Computer Vision and Kalman Filter** [Vision + Filtering]
- **Following the logical path** [Motion Control]
- **Local Navigation and Implementation of the "Dumb" Thymio** [Local Navigation + Motion Control]

We individually worked on those and then added slowly one part with another and when the result was good, we added another part and so on. 


## 1. Initialization
### Connection to Thymio and libraries
The next cells will connect to Thymio and import all the libraries and the other folders that we will need

In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

CancelledError: 

In [2]:
import cv2

In [1]:
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
#TO VERIFY IF WE STILL NEED THIS ONE!!!
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import* 
from optimal_path import* 
from KalmanFilter import KalmanFilter



AttributeError: module 'tdmclient' has no attribute 'notebook'

### Variables and constants


In [4]:
#Initialize the initial grid and position
#start , goal , grid = "init_map"
#start_pos = (0,0)
#goal = (43,33)
map_lenght = 5
nb_of_square_by_side = 50
current = 1

#variables
global motor_left_target, motor_right_target



## 2. Useful functions
Functions in order to use Thymio's actuators and sensors easily

In [5]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

In [6]:
@tdmclient.notebook.sync_var
def proxi():
    """
    Returns the proximity values of the Thymio 
    """
    global prox_horizontal
    return prox_horizontal

## 3. Current goal update
To be completed by Antoine

Sections that go into a bit more detail regarding the implementation and are accompanied by the code required to
execute the different modules independently. What is important is not to simply describe the code, which should
be readable, but describe what is not in the code: the theory behind, the choices made, the measurements, the
choice of parameters etc. Do not forget to cite your sources ! You can of course show how certain modules work in
simulation here, or with pre-recorded data. 

Next cell is to create an instance Robot and Map 
  

In [ ]:
global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
#pourcentage_reduc = 1

In [7]:
George = Robot()
# nb_of_square_by_side is the nb square that we want regarding x
Lausanne = Map(map_lenght, nb_of_square_by_side)
KF=KalmanFilter(0.1, [0, 0])

In [ ]:
#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)

print('Hello World')


Init part
Global control 

--> Optimal path calculation

In [4]:
# Prend une première image
ret, frame=VideoCap.read()

# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_pourcentage(frame)

# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())
pos_robot = George.get_pos()
while pos_robot < 0:
    pos_robot, angle = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....

George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("position", )


mask = mask_map_init(frame)
cv2.imshow("mask", mask)

Lausanne.init_grid(mask)

# affiche la grille
cv2.imshow("final secure grid", Lausanne.get_map())



######
# Compute the optimal path and visited nodes
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), Lausanne.get_map())

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  Lausanne.get_map(),  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())

1

## 4. Vision
To be completed by Xavier

Sections that go into a bit more detail regarding the implementation and are accompanied by the code required to
execute the different modules independently. What is important is not to simply describe the code, which should
be readable, but describe what is not in the code: the theory behind, the choices made, the measurements, the
choice of parameters etc. Do not forget to cite your sources ! You can of course show how certain modules work in
simulation here, or with pre-recorded data. 

## 5. To follow the logical path


The function takes continuously as parameters the position of the robot and its angle which are given by the Vision module. The angle is in the range of [-pi, pi]. It also takes as parameters the continuously updated goals that are given by the Optimal Path module.

We implement a function that computes the error in position (which is the distance between the goal and the robot) and the error in angle (We are inspired by the function described in https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python to determine the angle of the goal. We changed it to be optimal to the variation we want from the robot (all explained in the comments)).

As the angle is computed as a difference between the goal and the robot, if it's positif, then the robot should turn to the left, and if it's negatif, it should turn to the right, which is why we substract the angular error from the right motor speed, and add it to the left motor speed. 

In a more general way, the function "move_to_position" implements a PD (Proportionnal Derivative Controller), by adding a certain speed computed via the remaining distance (multiplied by an experimental coefficient) and adding/substracting an angular speed (multiplied by another experimental coefficient) to the motor's speed. These are added to some basic speed, because otherwise the speed of the robot will be almost zero when it reaches the updated goals. 


## 6. Local Navigation
To be completed by me

Sections that go into a bit more detail regarding the implementation and are accompanied by the code required to
execute the different modules independently. What is important is not to simply describe the code, which should
be readable, but describe what is not in the code: the theory behind, the choices made, the measurements, the
choice of parameters etc. Do not forget to cite your sources ! You can of course show how certain modules work in
simulation here, or with pre-recorded data. 

## 7. Main loop

A section which is used to run the overall project and where we can see the path chosen, where the system
believes the robot is along the path before and after filtering etc… This can also be done in a .py file if you prefer.
Just make sure to reference it in the report so we can easily have a look at it. 

In [7]:

kalman_bool = False


print('while start')
while True:#(George.get_pos() != George.get_goal()):
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Reupere la position et angle --> XAV
    #x_robot, y_robot,angle_robot = update(frame, Lausanne.get_pourcentage(), kalman_bool)
    pos_robot,angle_robot = update(frame, Lausanne.get_pourcentage())
    #KF.kalman_update(pos, speed)
    George.set_pos(pos_robot)
    George.set_angle(angle_robot)
    print ('position',pos_robot ,angle_robot)
    
    
    # Current goal update --> ANTOINE
    
    current = path_update(George.get_pos(), George.get_err_pos(), George.get_path(), George.get_current())
    
    # Check obstacle --> ALICIA
    
    # Faire avancer --> NOUR
    
    
    proximity = proxi()
    
        #LOCAL NAVIGATION
    if move:
        #Thymio is avoiding obstacles
        speed_l, speed_r, move = avoid_obstacle(prox_horizonta=proximity) 
    else:
        #Thymio is following the optimal path
        speed_l, speed_r = move_to_position(x_robot, y_robot, angle_robot, 152,262)
        #Thymio is checking if there's an obstacle in front of it
        move = check_cars(prox_horizonta=proximity)
        print (speed_l, speed_r)
        
    motors(speed_l, speed_r) 
    
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
     
    sleep(0.1)

print("The End")
    


Hello World
while start
position 0 0 0
0 0
debut 0 0 0
dist_debut 302.89932320822373
v 605.7986464164475
w 41.803931990020935
speed, 41 -41
41 -41
position 0 0 0
0 0
debut 0 0 0
dist_debut 302.89932320822373
v 605.7986464164475
w 41.803931990020935
speed, 41 -41
41 -41


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte

## 8. "Dumb" Thymio
We have implemented this code in another Thymio. The "Dumb" Thymio is the robot which will challenge the local avoidance of our main Thymio.

In this part, the "dumb" Thymio will follow a line until it sees an object in front of it. In this case, it stops (and screams) until the object is removed. We start the robot by pushing on its center button and we stop it the same way. 

We have chosen to flash this code into the robot as there was no need to have communications with the computer for this part. We used the exercices from week 3 to help us for this code. 

We chose to keep in memory the last turn in order 

You can find the schematic pseudo code here:
<img src="image/pseudo_dumb.jpg" alt="Drawing" style="width: 400px;"/>

To connect to the "dumb" Thymio:

In [4]:
import tdmclient.notebook
await tdmclient.notebook.start()

The code: 

In [12]:
%%run_python

# Implementation of **Dumb Thymio**
# 
# Author: Alicia Mauroux, Robotic MA1, Fall 2021

#constants and variables
white = 900
GOAL = 100
PROX = 2000
PROX_SIDE = 1600

lost = False
last_turn = False #False = left; True = Right
w_le = 0
w_ri = 0
state = 0

timer_period[0] = 100   # 100ms sampling time

######################
#     FUNCTIONS
#####################
@onevent
def button_center():
    """
    start/stop the robot by pushing on the center button 
    
    global state is the state of our Thymio (1 -> Thymio is running, 0 -> Thymio is sleeping)
    """
    global state
    if button_center == 1:
        state = 1 if state==0 else 0

def ground_white(white_threshold):
    """
    Tests whether the two (or one of the two) ground sensors have seen white
    
    param white_threshold: threshold starting which it is considered that the sensor saw white
    global turn_left means that Thymio is turning left
    global lost means that Thymio lost its lign
    global w_le is the left prox_ground value
    global w_ri is the right prox_ground value
    global last_turn indicate which was the last turn of Thymio 
    """
    global turn_left, prox_ground_reflected, lost, w_le, w_ri, last_turn
    
    w_le = prox_ground_reflected[0] 
    w_ri = prox_ground_reflected[1] 
    
    #white on the left --> turn right then to keep the track
    if (w_le > white_threshold):
        if (w_ri < white_threshold):
            turn_left = False
            lost = False
            last_turn = True #Last turn was to the right
            return True
    #white on both direction --> you lost the track! turn back
        else:
            lost = True
            turn_left = False
            return True    
    else:
    #white on the right --> turn left then to keep the track
        if(w_ri > white_threshold):
            lost = False
            turn_left = True
            last_turn = False #Last turn was to the left
            return True
    #following the lign!
        else:
            lost = False
            turn_left = False
            return False

def test_object(prox_threshold,prox_threshold_side):
    """
    Tests whether the front proximity sensors saw an object on Thymio's way. If there's something, a sound will be played.
    
    param prox_threshold: threshold starting which it is considered that the sensor saw an object
    param prox_threshold_side: threshold starting which it is considered that the side sensor saw an object
    """
    global prox_horizontal
    
    if (prox_horizontal[2]>prox_threshold):
        nf_sound_play(5)
        return True
    elif (prox_horizontal[1]>prox_threshold_side):
        nf_sound_play(5)
        return True
    elif (prox_horizontal[3]>prox_threshold_side):
        nf_sound_play(5)
        return True
    else:
        return False
        
def follow_lign(goal):
    """
    Will follow the lign by turning left/right each time the right/left ground sensor is seeing white
    
    param goal: It's the sensor's value of the black lign that Thymio is following 
    param prox_threshold_side: threshold starting which it is considered that the side sensor saw an object
    global lost: means that Thymio lost its lign
    global w_le: is the left prox_ground value
    global w_ri: is the right prox_ground value
    global last_turn: indicate which was the last turn of Thymio 
    """
    global turn_left, lost, last_turn, motor_left_target, motor_right_target
    
    SPEED0 = 200
    SPEED_LOST = SPEED0//2
    TURN = 150

    if(test_object(PROX,PROX_SIDE)):
        l_speed = 0
        r_speed = 0
    else:
        l_speed = SPEED0
        r_speed = SPEED0

        if (ground_white(white)):
            if(turn_left):
                l_speed = l_speed - TURN
                r_speed = r_speed + TURN
            elif(lost):
                if(last_turn):
                    l_speed = SPEED_LOST
                    r_speed = -SPEED_LOST
                else:
                    l_speed = -SPEED_LOST
                    r_speed = SPEED_LOST
            else:
                l_speed = l_speed  + TURN
                r_speed = r_speed - TURN

    motor_left_target = l_speed
    motor_right_target = r_speed

##############################
#  MAIN LOOP OF DUMB THYMIO
##############################
@onevent 
def timer0():
    
    global motor_left_target, motor_right_target
    
    if state:
        follow_lign(GOAL)
    else:
        motor_left_target = 0
        motor_right_target = 0

## 9. Conclusion

In conclusion, it was a very nice project and it went well. We were a good team and it was easy to work with each other. We had some issues with the part regarding the camera but the problem has been solved. 

## 10. Sources
- https://pypi.org/project/tdmclient/ 
- http://wiki.thymio.org/en:thymioapi
- https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python
- Exercices from the course (Basics of mobile robotics [MICRO-452])
- Control your Thymio in Python




